<a target="_blank" href="https://colab.research.google.com/github/ashater/sae/blob/master/ActivationDetectTwoPass.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip install nnsight==0.4.0 sae-lens

In [ ]:
!pip install --upgrade --force-reinstall numpy scipy scikit-learn

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 27.3 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uninstalled threadpoolctl-3.6.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  A

In [ ]:
#!pip install --upgrade --force-reinstall numpy

In [ ]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

import huggingface_hub


# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:0", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 6

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:0'
)
# Load the SAE weights using safetensors
# from safetensors import safe_open
# with safe_open("/home/ashater/work/sae.safetensors", framework="pt") as st:
#     w_dec = st.get_tensor("W_dec").to('cuda:1')





In [ ]:

from huggingface_hub import hf_hub_download

# Define the repository ID and filename
repo_id = "jbloom/Gemma-2b-Residual-Stream-SAEs"
filename = "sae_weights.safetensors"

# Download the file


# filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.6.hook_resid_post_16384")

#Layer 12
#filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.12.hook_resid_post_16384")

# Layer 6
filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr")

# Print the downloaded file path
print(filepath)

# Print the downloaded file path
print(filepath)

/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors
/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors


In [ ]:
# Load the SAE weights using safetensors
from safetensors import safe_open
with safe_open(filepath, framework="pt") as st:
    w_dec = st.get_tensor("W_dec").to('cuda:0')

In [ ]:
import torch
#torch.cuda.is_available()

#torch.cuda.device_count()
#len(docs)
torch.cuda.set_device(0)
torch.cuda.current_device()


0

In [ ]:
#sae.W_dec == w_dec

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(w_dec[471,:], sae.W_dec).topk(100).indices

tensor([  471,  4579, 11129, 11728,  6446,  6510, 14611,  7687,  5411,   891,
        15780,  7372,   675, 13371,   713, 15976,  1076,  5419,   976, 13837,
        10036,  2817,  6495,   154,  2287, 15131, 16291,  3466,  2111, 14557,
         9715,  8481,  8099,  8567, 15023, 11584,  3869,  7062, 15369,  3864,
        14291,  5239,  7807,  4189,  4815, 13685, 10031,  2126,  5936, 14224,
        15698, 14444, 10105, 10185,  5407,  6545,  8586,  8753,  6027, 13939,
        11269,  9963,  8461, 15760,  2312,   544,  9188, 11808,  8498,   354,
         4989,  7682, 11355,  4828, 14022,  3586,  8306,   954, 12170,   167,
        11255,  9691,  6426, 11718, 14749,  7608, 14153,  3971, 14416, 13584,
         3900, 13970,  8222, 14874,  8523,  9004, 12520, 15384,  1805,   575],
       device='cuda:0')

Parameters

In [ ]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
# feature = 3629
scale = 1 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 200  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
# vals = ['positive', 'neutral', 'negative']
# samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

## Main code prompt


In [ ]:
full_res = []
#v = range(0,10)
#for ismpl in v:


    # prompt = """<start_of_turn>user\nCan you rate  the following sentnece as a financial credit sentiment, very positive,
    # somewhat positive, neutral, somewhat negative,
    # very negative? """ + ismpl + '<end_of_turn>\n<start_of_turn>model\nThe sentiment of the setence is "X"'


prompt = "Credit Risk"
# prompt = """Credit Risk Related"""



#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
# positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
# print(positions)
#with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=False, scan=False, validate=False, temperature=0.0) as gen:
with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # vector steering section
    # for feature_i in topics:
    # vector = sae.W_dec[[feature]]
    # vector = vector / vector.norm()
    # vector = vector * scale
    # setting vector into layer 2
    # for position in positions:
    #   model.model.layers[2].output[0][:, position] = vector

    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[6].output.save()

features = sae.encode(resid[0])

summed_activations = features.abs().sum(dim=1) # Sort by max activations

print(summed_activations)
summed_activations = features
top_activations_indices = summed_activations.topk(1000) # Get indices of top 20
print(top_activations_indices)
#top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    #print(model.tokenizer.batch_decode(out))
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    #print(prompt + "|"+ s + "|" + str(top_activations_indices))
    #print(prompt + "|"+ l +"\n" + "|" + str(top_activations_indices))
    #print(str(top_activations_indices))


tensor([[ 2.6437, 19.0839,  0.0000,  ...,  0.0000, 11.5041,  0.0000]],
       device='cuda:0', grad_fn=<SumBackward1>)
torch.return_types.topk(
values=tensor([[[51.1168, 48.4640, 45.8361,  ..., 18.9716, 18.9661, 18.9428],
         [13.9263,  3.4869,  2.9293,  ...,  0.0000,  0.0000,  0.0000],
         [12.2713,  5.2073,  2.2165,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[[ 3390, 15881,  5347,  ...,  8207,    15,  3380],
         [ 5419, 10035,   471,  ...,   769,   770,   771],
         [10870,   471, 12704,  ...,   899,   900,   901]]], device='cuda:0'))


## summed activations across tokens, finance features

In [ ]:
s

'" Assessment:\\n\\nSure, here\'s an overview of the credit risk assessment process and a step-by-step guide on how to conduct a credit risk assessment:\\n\\n**The credit risk assessment process outlines the following steps:**\\n\\n- Gathering and analyzing relevant financial and non-financial data on the borrower or debtor.\\n- Estimating the probability that the borrower will not repay the debt.\\n- Calculating the potential loss amount if the borrower defaults on the debt.\\n- Using the estimated loss amount to determine the credit risk score.\\n\\n**Step 1: Identifying relevant data sources**\\n\\n- Financial statements: Balance sheets, income statements, tax returns\\n- Credit reports: Moody\'s, S&P, Fitchratings\\n- Third-party data: Marketing trends, economic indicators, industry reports\\n\\n**Step 2: Gathering and analyzing financial data**\\n\\n- Calculate the borrower\'s credit score by summing the assets and liabilities and dividing by two.\\n- Analyze the borrower\'s finan

In [ ]:
 #model.tokenizer.decode(model.tokenizer.encode(prompt)[2])
summed_activations[0][0]


tensor([ 2.6437, 19.0839,  0.0000,  ...,  0.0000, 11.5041,  0.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

3

In [ ]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[True, False, False, False, False, False, False, False, False, False, False]

In [ ]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1139).indices
[g in idc for g in [10870]]

[True]

## Get features for each token

In [ ]:
idc = summed_activations.mean(dim=1).topk(5000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[False, True, False, False, False, False, False, True, False, False, False]

In [ ]:
prompt = "Credit Risk"
# prompt = """Credit Risk Related"""



#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
# positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
# print(positions)
#with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=False, scan=False, validate=False, temperature=0.0) as gen:
with model.generate(s, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # vector steering section
    # for feature_i in topics:
    # vector = sae.W_dec[[feature]]
    # vector = vector / vector.norm()
    # vector = vector * scale
    # setting vector into layer 2
    # for position in positions:
    #   model.model.layers[2].output[0][:, position] = vector

    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[6].output.save()

features = sae.encode(resid[0])

summed_activations = features.abs().sum(dim=1) # Sort by max activations

print(summed_activations)
summed_activations = features
top_activations_indices = summed_activations.topk(1000) # Get indices of top 20
print(top_activations_indices)
#top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    #print(model.tokenizer.batch_decode(out))
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    #print(prompt + "|"+ s + "|" + str(top_activations_indices))
    #print(prompt + "|"+ l +"\n" + "|" + str(top_activations_indices))
    #print(str(top_activations_indices))

tensor([[2.9072e+00, 1.9084e+01, 1.4612e-02,  ..., 0.0000e+00, 1.1504e+01,
         4.7678e-03]], device='cuda:0', grad_fn=<SumBackward1>)
torch.return_types.topk(
values=tensor([[[51.1153, 48.4625, 45.8345,  ..., 18.9746, 18.9655, 18.9496],
         [ 5.7439,  3.7488,  2.1953,  ...,  0.0000,  0.0000,  0.0000],
         [15.0482,  3.9331,  3.1006,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 5.0740,  2.7632,  1.9795,  ...,  0.0000,  0.0000,  0.0000],
         [12.1458,  1.8587,  1.7906,  ...,  0.0000,  0.0000,  0.0000],
         [ 4.9684,  3.5519,  3.1325,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[[ 3390, 15881,  5347,  ...,  8207,    15,  3380],
         [ 2307,  2745,  3586,  ...,   943,   944,   945],
         [ 9481, 15202, 11912,  ...,   497,   498,   499],
         ...,
         [12931, 10438, 10392,  ...,   962,   963,   964],
         [15857,  3464, 11795,  ...,   964,   965,   966],
         [ 5145,   3

In [ ]:
summed_activations.mean(dim=1).topk(100)

torch.return_types.topk(
values=tensor([[1.0333, 0.9966, 0.4354, 0.3795, 0.3715, 0.3587, 0.3500, 0.3427, 0.3153,
         0.3145, 0.2898, 0.2779, 0.2607, 0.2578, 0.2493, 0.2329, 0.2325, 0.2324,
         0.2226, 0.2218, 0.2217, 0.2182, 0.2171, 0.2158, 0.2156, 0.2107, 0.2093,
         0.2093, 0.2060, 0.2054, 0.2053, 0.2030, 0.2021, 0.2001, 0.1996, 0.1992,
         0.1972, 0.1947, 0.1907, 0.1905, 0.1866, 0.1863, 0.1860, 0.1855, 0.1846,
         0.1842, 0.1841, 0.1840, 0.1837, 0.1816, 0.1813, 0.1793, 0.1780, 0.1778,
         0.1771, 0.1751, 0.1748, 0.1742, 0.1742, 0.1739, 0.1728, 0.1722, 0.1721,
         0.1716, 0.1694, 0.1692, 0.1691, 0.1675, 0.1661, 0.1656, 0.1653, 0.1650,
         0.1645, 0.1633, 0.1632, 0.1630, 0.1630, 0.1628, 0.1620, 0.1619, 0.1618,
         0.1611, 0.1611, 0.1609, 0.1605, 0.1605, 0.1603, 0.1596, 0.1596, 0.1594,
         0.1592, 0.1590, 0.1580, 0.1578, 0.1578, 0.1578, 0.1577, 0.1575, 0.1573,
         0.1571]], device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor(

In [ ]:
summed_activations.sum(dim=1).topk(100)

torch.return_types.topk(
values=tensor([[230.4269, 222.2342,  97.0888,  84.6276,  82.8532,  79.9836,  78.0472,
          76.4186,  70.3228,  70.1429,  64.6290,  61.9629,  58.1381,  57.4967,
          55.5992,  51.9326,  51.8386,  51.8215,  49.6356,  49.4656,  49.4347,
          48.6507,  48.4054,  48.1268,  48.0759,  46.9772,  46.6774,  46.6709,
          45.9280,  45.8145,  45.7785,  45.2706,  45.0675,  44.6199,  44.5068,
          44.4271,  43.9781,  43.4156,  42.5208,  42.4807,  41.6044,  41.5437,
          41.4669,  41.3699,  41.1685,  41.0796,  41.0643,  41.0246,  40.9684,
          40.4949,  40.4266,  39.9741,  39.6888,  39.6510,  39.4916,  39.0397,
          38.9847,  38.8553,  38.8369,  38.7890,  38.5328,  38.3942,  38.3734,
          38.2650,  37.7752,  37.7361,  37.7148,  37.3561,  37.0442,  36.9237,
          36.8554,  36.7973,  36.6816,  36.4095,  36.3920,  36.3465,  36.3430,
          36.3024,  36.1365,  36.1113,  36.0835,  35.9357,  35.9259,  35.8790,
          35.7891,  